<a href="https://colab.research.google.com/github/lydiamao/pytorch-learning_colab/blob/main/lydia_selflearning03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 #1 加载必要的库
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim
from torchvision import datasets,transforms

In [ ]:
#2 定义超参数
BATCH_SIZE=32 #每批处理的数据
DEVICE=torch.device("cuda") #用GPU
EPOCHS=20 #训练数据集的轮次，把一个数据集循环运行几轮

In [ ]:
#3 构建pipeline，对图像做处理
pipeline=transforms.Compose([
    transforms.ToTensor(), #将图片转化成tensor
    transforms.Normalize((0.1307,),(0.3081,)) # 正则化，模型出现过拟合现象时，降低模型复杂度
])

In [ ]:
#4 下载，加载数据
from torch.utils.data import DataLoader

#下载数据集
train_set=datasets.MNIST("data",train=True,download=True, transform=pipeline)

test_set=datasets.MNIST("data",train=False, download=True, transform=pipeline)

#加载数据
train_loader=DataLoader(train_set,batch_size=BATCH_SIZE,shuffle=True)

test_loader=DataLoader(test_set,batch_size=BATCH_SIZE,shuffle=True)

In [ ]:
#插入代码，显示MNIST中的图片
with open("./data/MNIST/raw/train-images-idx3-ubyte","rb") as f:
  file=f.read()

/content/data/MNIST/raw/train-images-idx3-ubyte

In [ ]:
import os
print(os.getcwd())

/content


In [ ]:
image1=[int(str(item).encode('ascii'),16)for item in file[16:16+784]]
print(image1)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 24, 24, 24, 294, 310, 373, 38, 358, 597, 583, 295, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 48, 54, 148, 340, 368, 595, 595, 595, 595, 595, 549, 370, 595, 578, 405, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 73, 568, 595, 595, 595, 595, 595, 595, 595, 595, 593, 147, 130, 130, 86, 57, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24, 537, 595, 595, 595, 595, 595, 408, 386, 583, 577, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 128, 342, 263, 595, 595, 517, 17, 0, 67, 340, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 1, 340, 595, 1

In [ ]:
import cv2
import numpy as np

image1_np=np.array(image1,dtype=np.uint8).reshape(28,28,1) #uint8无符号整型
print(image1_np.shape) #图片的高宽以及通道

(28, 28, 1)


In [ ]:
#图片保存
import matplotlib.pyplot as plt

cv2.imwrite("digit.jpg",image1_np)

True

In [ ]:
#5 构建网络模型
class Digit(nn.Module):
  def __init__(self):
    super().__init__()
    #定义卷积层
    self.conv1=nn.Conv2d(1,10,5) #1:灰度图片的通道，10:输出通道，5:kernel
    self.conv2=nn.Conv2d(10,20,3) #10:输入通道，20:输出通道，3:kernel
    #定义全连接层
    self.fc1=nn.Linear(20*10*10,500) #20*10*10:输入通道，500:输出通道
    self.fc2=nn.Linear(500,10) #500:输入通道，10:输出10类
  

    #定义前向传播
  def forward(self,x):
    input_size=x.size(0) #一批数据输入进模型的格式(张量形式)是：batch_size x 1 (图片的通道，灰度) x 28 x28 
                          #拿到batch_size
    x=self.conv1(x) #输入：batch_size*1*28*28, 输出：batch*10*24*24 (28-5+1=24)
    x=F.relu(x) #激活层，保持shape不变，输出：batch*10*24*24
    x=F.max_pool2d(x,2,2) #池化层，输入batch*10*24*24，输出：batch*10*12*12
    
    #调用第二个卷积层
    x=self.conv2(x) #输入：batch*10*12*12，输出：batch*20*10*10 (12-3+1=10)
    x=F.relu(x)

    x=x.view(input_size,-1) #拉平flatten，-1 自动计算维度，20*10*10=2000

    #送入全连接层
    x=self.fc1(x) #输入：batch*2000，输出：batch*500
    x=F.relu(x) #输出：batch*500

    #第二层全连接层
    x=self.fc2(x) #输入：batch*500，输出：batch*10

    #分类
    output=F.log_softmax(x,dim=1) #计算分类后，每个数字的概率值
    return output



In [ ]:
#6 定义优化器
model = Digit().to(DEVICE)

optimizer=optim.Adam(model.parameters()) #优化器，更新模型的参数，使得测试和训练的结果达到最优值

In [ ]:
#7 定义训练方法
def train_model(model,device,train_loader,optimizer,epoch):
  #模型训练
  model.train()
  for batch_index,(data,target) in enumerate(train_loader):
    #部署到DEVICE上去
    data,target=data.to(device),target.to(device)
    #梯度初始化为0
    optimizer.zero_grad()
    #训练后的结果
    output=model(data)
    #计算损失
    loss=F.cross_entropy(output,target)
    #反向传播
    loss.backward()
    #参数优化
    optimizer.step()
    if batch_index % 3000 == 0:
      print("Train Epoch : {} \t Loss : {:.6f}".format(epoch,loss.item())) #每3000张图片打印一次loss，.6f：保留小数点后六位

In [ ]:
#8 定义测试方法
def test_model(model,device,test_loader):
  #模型验证
  model.eval()
  #正确率
  correct=0.0
  #测试损失
  test_loss=0.0
  #开始测试
  with torch.no_grad(): #不会计算梯度，也不会进行反向传播
    for data,target in test_loader:
      #部署到device上
      data,target=data.to(device),target.to(device)
      #测试数据
      output=model(data)
      #计算测试损失
      test_loss+=F.cross_entropy(output,target).item()
      #找到概率值最大的下标
      pred=output.max(1,keepdim=True)[1] #1:横轴； 也可以写成：pred=output.argmax(dim=1); [1]:索引 [0]:值
      #累计正确的值
      correct/=pred.eq(target.view_as(pred)).sum().item()
  #计算平均损失值
  test_loss /= len(test_loader.dataset)
  print("Test-Average loss : {:4.4f},Accuracy:{:.3f}\n".format(test_loss,100*correct/len(test_loader.dataset)))


In [ ]:
#9 调用 方法7/8
for epoch in range(1,EPOCHS+1):
  train_model(model,DEVICE,train_loader,optimizer,epoch)
  test_model(model,DEVICE,test_loader)

Train Epoch : 1 	 Loss : 0.000003
Test-Average loss : 0.0023,Accuracy:0.000

Train Epoch : 2 	 Loss : 0.000041
Test-Average loss : 0.0033,Accuracy:0.000

Train Epoch : 3 	 Loss : 0.000021
Test-Average loss : 0.0029,Accuracy:0.000

Train Epoch : 4 	 Loss : 0.000282
Test-Average loss : 0.0032,Accuracy:0.000

Train Epoch : 5 	 Loss : 0.000007
Test-Average loss : 0.0027,Accuracy:0.000

Train Epoch : 6 	 Loss : 0.000007
Test-Average loss : 0.0039,Accuracy:0.000

Train Epoch : 7 	 Loss : 0.000000
Test-Average loss : 0.0034,Accuracy:0.000

Train Epoch : 8 	 Loss : 0.000001
Test-Average loss : 0.0031,Accuracy:0.000

Train Epoch : 9 	 Loss : 0.000000
Test-Average loss : 0.0033,Accuracy:0.000

Train Epoch : 10 	 Loss : 0.000000
Test-Average loss : 0.0045,Accuracy:0.000

Train Epoch : 11 	 Loss : 0.000000
Test-Average loss : 0.0044,Accuracy:0.000

Train Epoch : 12 	 Loss : 0.000000
Test-Average loss : 0.0050,Accuracy:0.000

Train Epoch : 13 	 Loss : 0.000000
Test-Average loss : 0.0033,Accuracy:0.